In [ ]:
# This notebook generates barplot with evaluation metrics for all groups specified in groups_eval variable. 


In [ ]:
basic_metrics = {('wtkappa', 'trim_round'): [0.7],
                 ('corr', 'trim'): [0.7],
                 ('SMD', 'trim_round'): [0.1, -0.1],
                 ('SMD', 'trim'): [0.1, -0.1],
                 ('R2', 'trim'): []}

colprefix = 'scale' if use_scaled_predictions else 'raw'
metrics = dict([('{}.{}_{}'.format(k[0], colprefix, k[1]), v) for k,v in basic_metrics.items()])

for group in groups_eval:
    display(Markdown('### Evaluation by {}'.format(group)))
    df_eval_group = pd.read_csv(join(output_dir, '{}_eval_by_{}.csv'.format(experiment_id, group)), index_col=0)
    df_eval_group.index.name = group
    df_eval_group.reset_index(inplace=True)
    
    # Define the order of the bars: put 'All data' first and 'No info' last.
    group_levels = list(df_eval_group[group])
    group_levels = [level for level in group_levels if level != 'All data']
    if 'No info' in group_levels:
        bar_names = ['All data'] + [level for level in group_levels if level != 'No info'] + ['No info']
    else:
        bar_names = ['All data'] + group_levels

    fig = plt.figure()
    fig.set_size_inches(10, len(metrics)*4)
    with sns.axes_style('white'):
        for i, metric in enumerate(sorted(metrics.keys())):
            df_plot = df_eval_group[[group, metric]]
            ax = fig.add_subplot(len(metrics), 1, i + 1)
            for lineval in metrics[metric]:
                ax.axhline(y=float(lineval), linestyle='--', linewidth=0.5, color='black')
            sns.barplot(df_plot[group], df_plot[metric], color='grey', ax=ax, order = bar_names)
            ax.set_xticklabels(bar_names, rotation=90) 
            ax.set_xlabel('')
            ax.set_ylabel('')
            ax.set_title('{} by {}'.format(metric, group))
    plt.tight_layout(h_pad=1.0)
    plt.savefig(join(figure_dir, '{}_eval_by_{}.svg'.format(experiment_id, group)))
    plt.show()